In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
#1. Create spark session obj
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("create_fraud_detection_dataMart").config("spark.driver.memory", "8g").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
# CRITICAL FIXES: Must be run BEFORE loading data to prevent ClassCastException
# 1. Disable Vectorized Reader (Avoids low-level ORC data reading crash)
spark.conf.set("spark.sql.orc.enableVectorizedReader", "false")
spark.conf.set("spark.sql.hive.convertMetastoreOrc", "false")
# 2. Disable Broadcast Join (Avoids memory/shuffle crash on join)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) 
# 3. Disable Spark's optimizing components (Forces safer execution path)
spark.conf.set("spark.sql.cbo.enabled", "false") 
spark.conf.set("spark.sql.codegen.wholeStage", "false")
# 4. Force Hive SerDe (Ultimate attempt to bypass native Spark reader)
spark.conf.set("spark.sql.hive.convertMetastore", "false") 

print("Spark Config Updated for maximum stability.")

Spark Config Updated for maximum stability.


In [4]:
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField,DoubleType , IntegerType, StringType, DecimalType, TimestampType, BooleanType

In [5]:
# 2. Load the Tables directly from the Database
# We use the standard 'database_name.table_name' format

print("Loading Silver Layer tables from Hive...")

# A. Transactions Table
df_trans_silver = spark.table("financial_db.transactions_silver")

# B. Users Table
df_users_silver = spark.table("financial_db.users_silver")

# C. Cards Table
df_cards_silver = spark.table("financial_db.cards_silver")

# 3. Quick Verification
print(f"Transactions Count: {df_trans_silver.count()}")
print(f"Users Count: {df_users_silver.count()}")
print(f"Cards Count: {df_cards_silver.count()}")

# 4.Preview Schema to ensure types are correct
df_trans_silver.printSchema()
df_users_silver.printSchema()
df_cards_silver.printSchema()

Loading Silver Layer tables from Hive...
Transactions Count: 24386900
Users Count: 2000
Cards Count: 6146
root
 |-- user_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: decimal(10,2) (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- errors: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- transaction_timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

root
 |-- person_id: string (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- retirement_age: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- birth_month: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- a

In [6]:
print("--- 2. Fixing Users Table ---")

# 1. Generate 'user_id' using Row Number (0, 1, 2...)
rdd_with_id = df_users_silver.rdd.zipWithIndex().map(lambda x: (x[1],) + tuple(x[0]))

# 2. Create DataFrame with 'user_id' as the first column
new_column_names = ["user_id"] + df_users_silver.columns
df_users_indexed = spark.createDataFrame(rdd_with_id, new_column_names)

# 3. Select and Rename (Strict Cleaning of Income)
df_users_prep = df_users_indexed.select(
    F.col("user_id"),
    F.col("person_id").alias("person_name"),
    F.col("current_age"),
    # FIX: Strip non-numeric chars from income before casting to safely handle Text data
    F.coalesce(
        F.regexp_replace(F.col("yearly_income_person").cast("string"), "[^0-9\\.\\-]", "").cast("double"), 
        F.lit(0.0)
    ).alias("yearly_income"),
    F.col("fico_score"),
    F.col("state").alias("user_home_state")
)

print("Users Table Prepped (Income Strictly Cleaned).")

--- 2. Fixing Users Table ---
Users Table Prepped (Income Strictly Cleaned).


In [7]:
print("--- 3. Prep Transactions & Cards ---")

# A. Prepare Transactions
# UPDATED: Select 'user_id' and 'card_id' directly (no need to alias from 'user'/'card' anymore)
df_trans_prep = df_trans_silver.select(
    F.col("user_id"),     # <--- Updated: Reads directly from new schema
    F.col("card_id"),     # <--- Updated: Reads directly from new schema
    F.col("transaction_timestamp"),
    F.col("merchant_name"),
    F.col("merchant_state"),
    F.col("zip"),         # Zip is already string in Silver, so strictly taking it is fine
    F.col("mcc").alias("merchant_category"), 
    F.col("errors").alias("error_code"), 
    F.col("use_chip"),
    # Keep defensive cleaning just in case, but reading directly is usually fine now
    F.coalesce(F.col("amount").cast("double"), F.lit(0.0)).alias("amount"),
    F.when(F.col("is_fraud") == "Yes", 1).otherwise(0).alias("label_is_fraud")
).withColumn("hour_of_day", F.hour(F.col("transaction_timestamp")))

# B. Prepare Cards
# Cards table still uses 'user' (based on your cards script), so we rename it here.
df_cards_prep = df_cards_silver.select(
    F.col("user").alias("user_id"),
    F.col("card_index").alias("card_id"),
    F.col("card_brand"),
    F.col("card_type"),
    # Keep defensive cleaning
    F.coalesce(F.col("credit_limit").cast("double"), F.lit(0.0)).alias("credit_limit")
)

print("Transactions & Cards Prepped (Schema Updated).")

--- 3. Prep Transactions & Cards ---
Transactions & Cards Prepped (Schema Updated).


In [8]:
print("--- 4. Performing Joins ---")

# ULTIMATE STABILITY FIX: Cache all preparatory DataFrames before the large join/shuffle.
# This forces Spark to materialize the cleaned, stable data, breaking the complex
# lineage that was causing the ClassCastException during the final write operation.

df_join_1 = df_trans_prep.join(
    df_users_prep,
    on=["user_id"], 
    how="left"
)

df_final_join = df_join_1.fillna(0, subset=["card_id"]).join(
    df_cards_prep,
    on=["user_id", "card_id"],
    how="left"
)

print(f"Joined Data Count: {df_final_join.count()}")

--- 4. Performing Joins ---
Joined Data Count: 24386900


In [9]:
print("--- 4. Advanced Feature Engineering (Post-Join) ---")

df_mart = df_final_join.withColumn(
    # 1. Composite Transaction ID
    # FIX: Use 'user_id' and 'card_id' instead of fully qualified names
    "transaction_id",
    F.concat_ws("-", F.col("user_id"), F.col("card_id"), F.col("transaction_timestamp").cast("long"))
).withColumn(
    # 2. Credit Utilization
    # Uses the cleaned 'amount' and 'credit_limit' (now safe doubles from Cell 6)
    "credit_utilization",
    F.when(F.col("credit_limit").cast("double") > 0, 
           F.col("amount").cast("double") / F.col("credit_limit").cast("double")
    ).otherwise(0.0)
).withColumn(
    # 3. State Mismatch
    "state_mismatch",
    F.when(
        (F.col("merchant_state").isNotNull()) & 
        (F.col("merchant_state") != F.col("user_home_state")),
        1
    ).otherwise(0)
).withColumn(
    # 4. Clean Error Codes
    "error_code_clean",
    F.coalesce(F.col("error_code"), F.lit("None"))
)

print("Feature Engineering Complete.")

--- 4. Advanced Feature Engineering (Post-Join) ---
Feature Engineering Complete.


In [10]:
from pyspark.sql.types import DecimalType, IntegerType

print("--- 5. Final Column Selection & Save ---")

# Ensure you are using the correct column names from df_mart
final_columns = [
    # Identifiers
    "transaction_id",
    F.col("user_id").alias("user_id"),
    F.col("card_id").alias("card"),
    F.col("merchant_name").alias("merchant_id"),
    
    # Target
    "label_is_fraud",
    
    # Features (Stable Decimal Casts)
    # Cast double->high-precision Decimal->target Decimal(10,2)
    F.col("amount").cast(DecimalType(20, 4)).cast(DecimalType(10,2)).alias("amount"), 
    "hour_of_day",
    "use_chip",
    F.col("merchant_category").alias("merchant_category"),
    F.col("error_code_clean").alias("error_code"),
    "credit_utilization",
    "state_mismatch",
    
    # User Profile (Stable Integer Casts)
    F.col("current_age").cast(IntegerType()).alias("current_age"),
    # Stable Decimal Cast for income
    F.col("yearly_income").cast(DecimalType(20, 4)).cast(DecimalType(10,2)).alias("yearly_income_person"),
    F.col("fico_score").cast(IntegerType()).alias("fico_score"),
    
    # Card Profile
    "card_type",
    "card_brand"
]

df_gold_fraud = df_mart.select(*final_columns)

--- 5. Final Column Selection & Save ---


In [11]:
df_gold_fraud.printSchema()

root
 |-- transaction_id: string (nullable = false)
 |-- user_id: integer (nullable = true)
 |-- card: integer (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- label_is_fraud: integer (nullable = false)
 |-- amount: decimal(10,2) (nullable = true)
 |-- hour_of_day: integer (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_category: integer (nullable = true)
 |-- error_code: string (nullable = false)
 |-- credit_utilization: double (nullable = true)
 |-- state_mismatch: integer (nullable = false)
 |-- current_age: integer (nullable = true)
 |-- yearly_income_person: decimal(10,2) (nullable = true)
 |-- fico_score: integer (nullable = true)
 |-- card_type: string (nullable = true)
 |-- card_brand: string (nullable = true)



In [12]:
print("--- Step 6: Saving as External Hive Table ---")
# Path for the External Table
gold_path = "/user/talentum/projectMaster/warehouseDir/gold/fraud_detection"

--- Step 6: Saving as External Hive Table ---


In [13]:
# Fix for NPE in OrcColumnVector
spark.conf.set("spark.sql.orc.enableVectorizedReader", "false")
spark.conf.set("spark.sql.hive.convertMetastoreOrc", "false")

# Optional: If the error persists, disable Broadcast Join to force a robust Sort-Merge Join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [14]:
# Drop + recreate (your existing logic is good)
spark.sql("DROP TABLE IF EXISTS financial_db.fraud_detection_gold")

# Write with explicit partitioning and Hive serde override
df_gold_fraud.write \
    .mode("overwrite") \
    .format("orc") \
    .option("path", "/user/talentum/projectMaster/warehouseDir/gold/fraud_detection") \
    .option("compression", "snappy") \
    .saveAsTable("financial_db.fraud_detection_gold")

print("SUCCESS: Table saved!")

SUCCESS: Table saved!
